In [1]:
import numpy as np
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.utils import save_image
from glob import glob

In [49]:
# class VoiceMFCC(Dataset):
#     """Voice MFCC spectra dataset."""

#     def __init__(self, csv_files, standardize=False):
#         """
#         Preconditions: csv files must contain matrices of the same dimension
#         Args:
#             csv_files (string or list): list of filenames/pathnames of csv files 
#                                         with MFCC spectrogram matrices
#             transform (callable, optional): Optional transform to be applied
#                                             on a sample.
#         """
#         # ensure csv_files is a list
#         if type(csv_files) == str:
#             csv_files = [csv_files]
        
#         # load csv files with the MFCC spectra into a 3D tensor
#         matrices = []
#         for f in csv_files:
#             matrix = np.loadtxt(f, delimiter=',', dtype=np.float32)
#             if standardize:
#                 matrix = (matrix - np.mean(matrix)) / np.std(matrix)
#             matrices.append(matrix)
#         self.X = torch.Tensor(matrices)
#         N, D, M = self.X.shape
#         self.X = self.X.view(N, 1, D, M) # insert channel dimension
        
#     def __len__(self):
#         return self.X.shape[0]

#     def __getitem__(self, idx):
#         sample = self.X[idx]

#         return sample 

In [49]:
class voice_face(Dataset):
    def __init__(self, csv_spectra, face_IDs, standardize=False):
        """
        Preconditions: csv files must contain matrices of the same dimension
        Args:
            csv_spectra (string or list): list of filenames/pathnames of csv files with spectrogram matrices
            face_ID (list or array):      list or array of corresponding face index numbers
            standardise (boolean):        whether to standardize the spectrograms
        """
        # ensure csv_spectra is a list
        if type(csv_spectra) == str:
            csv_spectra = [csv_spectra]
        
        # ensure same number of observations
        assert(len(csv_spectra) = len(list(face_IDs)))
        
        # load csv files with the spectra into a tensor
        matrices = []
        for f in csv_spectra:
            matrix = np.loadtxt(f, delimiter=',', dtype=np.float32)
            if standardize:
                matrix = (matrix - np.mean(matrix)) / np.std(matrix)
            matrices.append(matrix)
        self.X = torch.Tensor(matrices)
        N, D, M = self.X.shape
        self.X = self.X.view(N, 1, D, M) # insert channel dimension
        
        # save face_IDs
        self.y = torch.tensor(face_IDs)
        
    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [52]:
# import dataset
csv_files = glob("data/train_mfcc/*.csv")
# dataset = VoiceMFCC(csv_files, standardize=True)
dataset = VoiceMFCC(csv_files[:10], standardize=True)

In [53]:
# split dataset
TRAIN_SIZE = 9099
VALID_SIZE = 2000
TEST_SIZE = 2000
train_dataset, validation_dataset, test_dataset = torch.utils.data.random_split(dataset, 
                                                                                [TRAIN_SIZE, 
                                                                                 VALID_SIZE, 
                                                                                 TEST_SIZE])

In [54]:
# pipe data through a dataloader for batching
BATCH_SIZE = 20
dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [136]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.ModuleList(
            [
                nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1), 
                nn.ReLU(True),
                nn.MaxPool2d(2, stride=2), 
                nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
                nn.ReLU(True),
                nn.MaxPool2d(2, stride=2),
                nn.Conv2d(16, 64, kernel_size=3, stride=1, padding=1),
                nn.ReLU(True),
                nn.MaxPool2d(2, stride=2),
                nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)
            ]
        )
        
        self.decoder = nn.ModuleList(
            [
                nn.ConvTranspose2d(1, 64, kernel_size=3, stride=1, padding=1), 
                nn.ReLU(True),
                nn.ConvTranspose2d(64, 16, kernel_size=3, stride=2, padding=(1,0),output_padding = (1,0)), 
                nn.ReLU(True),
                nn.ConvTranspose2d(16, 8, kernel_size=3, stride=2, padding=(1,1),output_padding = (1,1)),
                nn.ReLU(True),
                nn.ConvTranspose2d(8, 1, kernel_size=3, stride=2, padding=(1,0), output_padding = (1,0)),
                nn.Tanh()
            ]
        )
        
        self.w_length = 5
        self.face_length = 128*128
        self.B = nn.Linear(self.w_length, self.face_length, bias=False)

    def forward(self, x):
#         print("shape start: {}".format(x.shape))
        
#         print("start encoder")
        for layer in self.encoder:
            x = layer.forward(x)
            print(x.shape)
        
        # collapse final feature map into a vector by taking average across time
        N, _, H, _ = x.shape
        w = x.mean(dim=3)
        w = w.view(N, H)
        
#         print("start decoder")     
        for layer in self.decoder:
            v = layer.forward(x)
            print(v.shape)
            
#         print("start face construction")
        f = self.B(w)
        
        return v, f, w

In [ ]:
ALPHA = 1
def my_loss(model_output,labels):
    voice_output, face_output = model_output
    
    voice_data, face_id = labels
    face_data = face_struct[face_id]
    
    loss = fid(face_output, face_data) + ALPHA * MSE(voice_output, voice_data)
    return loss

In [137]:
# model = autoencoder().cuda()
NUM_EPOCHS = 10
LEARNING_RATE = 1e-3
model = autoencoder()
MSE = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

In [110]:
# for data in dataloader:
#     temp = data
#     break

In [111]:
# temp.shape

torch.Size([20, 1, 40, 173])

In [ ]:
for epoch in range(NUM_EPOCHS):
    for batch in dataloader:
        # ===================forward=====================
        voice, face_id = batch
        voice_output, face_output = model(voice)
        loss = my_loss((voice_output,face_output), (voice,face_id))
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    if((epoch+1)%10 == 0):
        print('epoch [{}/{}], loss:{:.4f}'
              .format(epoch+1, NUM_EPOCHS, loss.data.item()))

torch.save(model.state_dict(), './model_state.pth')

In [ ]:
# Check that NN outputs the same size file as input
# FID for similarity between faces